In [1]:
import fxcmpy
import time
import numpy as np
import pandas as pd
import threading as thr
import re
from datetime import datetime
from playsound import playsound

In [2]:
TOKEN='bf7a2bd697cd4b6024f83aa3c1100123972830a7'
con = fxcmpy.fxcmpy(access_token=TOKEN, server='demo')
con.is_connected()

|WARNING|2021-05-07 01:32:52,808|Default account set to 4290918, to change use set_default_account().


True

In [4]:
def BUY(inst,risk,Sl,Tp):
    
    JPY_Pattern = re.compile(r'\w{3}/JPY')    
    JPY = JPY_Pattern.findall(inst)
    
    if JPY != None :
        con.subscribe_market_data('USD/JPY')
        JPY_LastPrice = int(con.get_last_price('USD/JPY')[0])
        lot = int((risk/JPY_LastPrice*10000)/Sl)
 
        order = con.create_market_buy_order(inst, lot)
        playsound('C:\\Users\\HAMED\\miniconda3\\envs\\fxcm\\Lib\\site-packages\\order.wav')
        ID = con.get_open_trade_ids()[-1]
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=True, rate=-(Sl/10))
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=False, rate=Tp/10)
        print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                '%s | %s |'  %(inst,timeframe) + ' ----->  BUY Position Opened')  
        
    else :
        lot = int((risk/Sl)*100)
        order = con.create_market_buy_order(inst, lot)
        playsound('C:\\Users\\HAMED\\miniconda3\\envs\\fxcm\\Lib\\site-packages\\order.wav')
        ID = con.get_open_trade_ids()[-1]
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=True, rate=-(Sl/10))
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=False, rate=Tp/10)
        print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                '%s | %s |'  %(inst,timeframe) + ' ----->  BUY Position Opened')     

In [5]:
def SELL(inst,risk,Sl,Tp):
    
    JPY_Pattern = re.compile(r'\w{3}/JPY')    
    JPY = JPY_Pattern.findall(inst)
        
    if JPY != None :
        con.subscribe_market_data('USD/JPY')
        JPY_LastPrice = int(con.get_last_price('USD/JPY')[0])
        lot = int((risk/JPY_LastPrice*10000)/Sl)
 

        order = con.create_market_sell_order(inst, lot)
        playsound('C:\\Users\\HAMED\\miniconda3\\envs\\fxcm\\Lib\\site-packages\\order.wav')
        ID = con.get_open_trade_ids()[-1]
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=True, rate=-(Sl/10))
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=False, rate=Tp/10)
        print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                '%s | %s |'  %(inst,timeframe) + ' ----->  SELL Position Opened') 
        
    else :
        lot = int((risk/Sl)*100)
        order = con.create_market_sell_order(inst, lot)
        playsound('C:\\Users\\HAMED\\miniconda3\\envs\\fxcm\\Lib\\site-packages\\order.wav')
        ID = con.get_open_trade_ids()[-1]
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=True, rate=-(Sl/10))
        con.change_trade_stop_limit(ID, is_in_pips=True,is_stop=False, rate=Tp/10)
        print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                '%s | %s |'  %(inst,timeframe) + ' ----->  SELL Position Opened')  

In [6]:
def UpTrend(timeframe, inst, lastBID_open,risk,Sl,Tp):
    data = con.get_candles(instrument=inst, period=timeframe, number=26)
    if data['askclose'][-1] > data['askhigh'][0] :
        print('> %s | ' %(datetime.now().strftime("%H:%M:%S"))+
                '%s | %s | ChikouSpan is Up -- BUY Order is Taking...'  %(inst,timeframe))
        BUY(inst,risk,Sl,Tp)
        
    else : 
        while ICHI(timeframe,inst)[0] == 'up':    
            data = con.get_candles(instrument=inst, period=timeframe, number=26)
            if  lastBID_open != data['bidopen'][-1]: 
                if data['askclose'][-1] > data['askhigh'][0] :
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S"))+
                          '%s | %s | ChikouSpan is Up -- BUY Order is Taking...'  %(inst,timeframe))
                    BUY(inst,risk,Sl,Tp)
                    break
                else :
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                          '%s | %s | Waiting for ChikouSpan Validation...'  %(inst,timeframe))
                    lastBID_open = data['bidopen'][-1]
            else :
                if timeframe == 'm1' :
                    time.sleep(60)
                elif timeframe == 'm5' :
                    time.sleep(300 - (int(datetime.now().strftime("%M"))%5)*60)
                elif timeframe == 'm15' :
                    time.sleep(900 - (int(datetime.now().strftime("%M"))%15)*60)
                elif timeframe == 'm30' :
                    time.sleep(1800 - (int(datetime.now().strftime("%M"))%30)*60)
                elif timeframe == 'H1' :
                    time.sleep(3600 - (int(datetime.now().strftime("%M"))%60)*60)

In [7]:
def DownTrend(timeframe, inst, lastBID_open,risk,Sl,Tp):
    data = con.get_candles(instrument=inst, period=timeframe, number=26)
    if data['bidclose'][-1] < data['bidlow'][0] :
        print('> %s | ' %(datetime.now().strftime("%H:%M:%S"))+
              '%s | %s | ChikouSpan is Down -- SELL Order is Taking...'  %(inst,timeframe))
        SELL(inst,risk,Sl,Tp)
        
    else : 
        while ICHI(timeframe,inst)[0] == 'down':    
            data = con.get_candles(instrument=inst, period=timeframe, number=26)
            if  lastBID_open != data['bidopen'][-1]: 
                if data['bidclose'][-1] < data['bidlow'][0] :
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S"))+
                          '%s | %s | ChikouSpan is Down -- SELL Order is Taking...'  %(inst,timeframe))
                    SELL(inst,risk,Sl,Tp)
                    break
                else :
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                          '%s | %s | Waiting for ChikouSpan Validation...'  %(inst,timeframe))
                    lastBID_open = data['bidopen'][-1]
            else :
                if timeframe == 'm1' :
                    time.sleep(60)
                elif timeframe == 'm5' :
                    time.sleep(300 - (int(datetime.now().strftime("%M"))%5)*60)
                elif timeframe == 'm15' :
                    time.sleep(900 - (int(datetime.now().strftime("%M"))%15)*60)
                elif timeframe == 'm30' :
                    time.sleep(1800 - (int(datetime.now().strftime("%M"))%30)*60)
                elif timeframe == 'H1' :
                    time.sleep(3600 - (int(datetime.now().strftime("%M"))%60)*60)

In [8]:
def Road(timeframe, inst,lastBID_open, risk, Sl,Tp, Kijunsen,crossNum): 
    number = 27    
    while True :
        data = con.get_candles(instrument=inst, period=timeframe, number=number)
        HstHigh_tk = np.array(data)[17:26,2].max()   
        LstLow_tk = np.array(data)[17:26,3].min()             
        HstHigh_kj = np.array(data)[:26,2].max()      
        LstLow_kj = np.array(data)[:26,3].min()
        Tenkensen_Old = (HstHigh_tk + LstLow_tk)/2 
        Kijunsen_Old = (HstHigh_kj + LstLow_kj)/2            
        if Tenkensen_Old < Kijunsen_Old : 
            print('> %s | ' %(datetime.now().strftime("%H:%M:%S"))+
                    '%s | %s | UpRoad(%s) -- Waiting for Kijunsen Diversion...'  %(inst,timeframe,crossNum))
            UpRoad= thr.Thread(target= Bull_Hunter, args=(timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum))
            UpRoad.start()
            break
                
        elif Tenkensen_Old > Kijunsen_Old : 
            print('> %s | ' %(datetime.now().strftime("%H:%M:%S"))+
                    '%s | %s | DownRoad(%s) -- Waiting for Kijunsen Diversion...'  %(inst,timeframe,crossNum))
            DownRoad= thr.Thread(target= Bear_Hunter, args=(timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum))
            DownRoad.start()
            break
                
        else :
            number = number + 1 

In [9]:
def Bear_Hunter (timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum):
    while True :
        if ICHI(timeframe,inst)[0] == 'down' or ICHI(timeframe,inst)[0] == 'road' :
            data = con.get_candles(instrument=inst, period=timeframe, number=26)
            if  lastBID_open != data['bidopen'][-1] :  
                HstHigh_kj = np.array(data)[:26,2].max()      
                LstLow_kj = np.array(data)[:26,3].min()
                KijunSen_new = (HstHigh_kj + LstLow_kj)/2       
                if KijunSen_new < Kijunsen:
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + 
                          '%s | %s | Cross(%s) -- Kjijunsen Shifted Down'  %(inst,timeframe,crossNum))
                    Chikou_Down_Thread = thr.Thread(target=DownTrend, args= (timeframe, inst, lastBID_open,risk,Sl,Tp))
                    Chikou_Down_Thread.start()                                 
                    break
                   
                else:
                    lastBID_open = data['bidopen'][-1]
                    
            else :
                if timeframe == 'm1' :
                    time.sleep(60)
                elif timeframe == 'm5' :
                    time.sleep(300 - (int(datetime.now().strftime("%M"))%5)*60)
                elif timeframe == 'm15' :
                    time.sleep(900 - (int(datetime.now().strftime("%M"))%15)*60)
                elif timeframe == 'm30' :
                    time.sleep(1800 - (int(datetime.now().strftime("%M"))%30)*60)
                elif timeframe == 'H1' :
                    time.sleep(3600 - (int(datetime.now().strftime("%M"))%60)*60)   
                
        else :
            print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + 
              '%s | %s | Cross(%s) Failed -- Waiting for Another Cross...'  %(inst,timeframe,crossNum)) 
            break    

In [10]:
def Bull_Hunter (timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum):
    while True :
        if ICHI(timeframe,inst)[0] == 'up' or ICHI(timeframe,inst)[0] == 'road' :
            data = con.get_candles(instrument=inst, period=timeframe, number=26)
            if  lastBID_open != data['bidopen'][-1] :       
                HstHigh_kj = np.array(data)[:26,2].max()      
                LstLow_kj = np.array(data)[:26,3].min()
                KijunSen_new = (HstHigh_kj + LstLow_kj)/2            
                if KijunSen_new > Kijunsen:
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + 
                      '%s | %s | Cross(%s) -- Kjijunsen Shifted Up'  %(inst,timeframe,crossNum))
                    Chikou_Up_Thread = thr.Thread(target=UpTrend, args= (timeframe, inst, lastBID_open,risk,Sl,Tp))
                    Chikou_Up_Thread.start()
                    break
                                
                else:
                    lastBID_open = data['bidopen'][-1]
                    
            else :
                if timeframe == 'm1' :
                    time.sleep(60)
                elif timeframe == 'm5' :
                    time.sleep(300 - (int(datetime.now().strftime("%M"))%5)*60)
                elif timeframe == 'm15' :
                    time.sleep(900 - (int(datetime.now().strftime("%M"))%15)*60)
                elif timeframe == 'm30' :
                    time.sleep(1800 - (int(datetime.now().strftime("%M"))%30)*60)
                elif timeframe == 'H1' :
                    time.sleep(3600 - (int(datetime.now().strftime("%M"))%60)*60)
    
        else :
            print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + 
              '%s | %s | Cross(%s) Failed -- Waiting for Another Cross...'  %(inst,timeframe,crossNum)) 
            break      

In [11]:
def ICHI(timeframe,inst) :
    data = con.get_candles(instrument=inst, period=timeframe, number=26)
    HstHigh_tk = np.array(data)[17:,2].max()   
    LstLow_tk = np.array(data)[17:,3].min()             
    HstHigh_kj = np.array(data)[:26,2].max()      
    LstLow_kj = np.array(data)[:26,3].min()
    TenkenSen = (HstHigh_tk + LstLow_tk)/2 
    KijunSen = (HstHigh_kj + LstLow_kj)/2
         
    if KijunSen < TenkenSen:
        ichi = 'up'
        return ichi , KijunSen , TenkenSen                             
                        
    elif KijunSen > TenkenSen:
        ichi = 'down'
        return ichi , KijunSen , TenkenSen                           
       
    else:
        ichi = 'road'
        return ichi , KijunSen , TenkenSen    

In [12]:
def Ichimoku(timeframe,inst,lastBID_open):
    while True:
        data = con.get_candles(instrument=inst, period=timeframe, number=26) 
        if  lastBID_open != data['bidopen'][-1] :
            HstHigh_tk = np.array(data)[17:,2].max()   
            LstLow_tk = np.array(data)[17:,3].min()           
            HstHigh_kj = np.array(data)[:26,2].max()      
            LstLow_kj = np.array(data)[:26,3].min()
            KijunSen = (HstHigh_kj + LstLow_kj)/2
            TenkenSen = (HstHigh_tk + LstLow_tk)/2
            
            if KijunSen < TenkenSen:
                ichi = 'up'
                return ichi , KijunSen , TenkenSen                                       
            
            elif KijunSen > TenkenSen:
                ichi = 'down'
                return ichi , KijunSen , TenkenSen                             
    
            else:
                ichi = 'road'
                return ichi , KijunSen , TenkenSen   
            
        else :
            
            if timeframe == 'm1':
                time.sleep(60)
            elif timeframe == 'm5' :
                time.sleep(300 - (int(datetime.now().strftime("%M"))%5)*60)
            elif timeframe == 'm15' :
                time.sleep(900 - (int(datetime.now().strftime("%M"))%15)*60)
            elif timeframe == 'm30' :
                time.sleep(1800 - (int(datetime.now().strftime("%M"))%30)*60)
            elif timeframe == 'h1' :
                time.sleep(3600 - (int(datetime.now().strftime("%M"))%60)*60)

In [13]:
def SwitchFounder(timeframe,inst,risk,Sl,Tp):
    road = []
    crossNum = 0
    while True:          
        lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
        Trend = Ichimoku(timeframe,inst,lastBID_open)
        if Trend[0] == 'down':
            
            while True:
                lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                Trend = Ichimoku(timeframe,inst,lastBID_open)
                if Trend[0] == 'up':
                    lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                    crossNum = crossNum + 1
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                          '%s | %s | Cross(%s) Found -- Waiting for Kijunsen Diversion...'  %(inst,timeframe,crossNum)) 
                    Kijunsen = Trend[1]
                    road.clear()
                    UpObj= thr.Thread(target= Bull_Hunter, args=(timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum))
                    UpObj.start()
                    break  
                    
                elif Trend[0] == 'road' : 
                    lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                    crossNum = crossNum + 1
                    print ('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + 
                             '%s | %s | RoadCross(%s) Found -- Waiting for Kijunsen Diversion...'  %(inst,timeframe,crossNum))
                    Kijunsen = Trend[1]
                    road.append(inst)
                    UpObj= thr.Thread(target= Bull_Hunter, args=(timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum))
                    UpObj.start()
                    break
   
                    
                else :
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + '%s | %s | NOT MATCH | ' %(inst,timeframe) + 
                          '%.2f' %(100 -((Trend[1]-Trend[2])*250)) + ' D' 
                  ' |  TK: %.4f' %Trend[2] + ' -- KJ: %.4f' %Trend[1])                            
            
        elif Trend[0] == 'up':
            
            while True:
                lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                Trend = Ichimoku(timeframe,inst,lastBID_open)
                if Trend[0] == 'down':
                    lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                    crossNum = crossNum + 1
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) +
                          '%s | %s | Cross(%s) Found -- Waiting for Kijunsen Diversion...'  %(inst,timeframe,crossNum)) 
                    Kijunsen = Trend[1]
                    road.clear()
                    DownObj= thr.Thread(target= Bear_Hunter, args=(timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum))
                    DownObj.start()
                    break 
                    
                elif Trend[0] == 'road' : 
                    lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                    crossNum = crossNum + 1
                    print ('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + 
                             '%s | %s | RoadCross(%s) Found -- Waiting for Kijunsen Diversion...'  %(inst,timeframe,crossNum))
                    Kijunsen = Trend[1]
                    road.append(inst)
                    DownObj= thr.Thread(target= Bear_Hunter, args=(timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum))
                    DownObj.start()
                    break
        
                else :
                    print('> %s | ' %(datetime.now().strftime("%H:%M:%S")) + '%s | %s | NOT MATCH | ' %(inst,timeframe) + 
                          '%.2f' %(100 -((Trend[2]-Trend[1])*250)) + ' U' 
                  ' |  TK: %.4f' %Trend[2] + ' -- KJ: %.4f' %Trend[1])
                                      
        else :
            if inst not in road :   
                road.append(inst) 
                crossNum = crossNum + 1
                lastBID_open = con.get_candles(instrument=inst, period=timeframe, number=1)['bidopen'][-1]
                Kijunsen = Trend[1]
                RoadObj = thr.Thread(target= Road, args= (timeframe, inst, lastBID_open,risk,Sl,Tp,Kijunsen,crossNum))
                RoadObj.start()

In [14]:
currency = ['GBP/JPY' , 'USD/JPY' , 'AUD/USD' , 'GBP/USD',
            'USD/CAD' , 'NZD/USD', 'EUR/USD', 'USD/CHF']
risk = 20
Sl = 200
Tp = 400
timeframe = 'm15'
print('Connecting to Server...\n')
while True :
    extime = int(datetime.now().strftime("%M"))
    if extime%15 == 0 :
        for i in range(int(len(currency))) :
            inst = currency[i]
            tradeObj = thr.Thread(target = SwitchFounder , args=(timeframe,inst,risk,Sl,Tp))
            tradeObj.start() 
        thr.activeCount() 
        print('Start Hunting...\n')
        break
    else :
        time.sleep(30)

Connecting to Server



35

> 01:34:18 | EUR/AUD | m1 | RoadCross(1) Found... > 01:34:18 | EUR/USD | m1 | RoadCross(1) Found... 

> 01:34:18 | AUD/NZD | m1 | RoadCross(1) Found... 
> 01:34:18 | EUR/JPY | m1 | RoadCross(1) Found... 
> 01:34:18 | USD/JPY | m1 | RoadCross(1) Found... 
> 01:34:26 | EUR/JPY | m1 | DownRoad(1) -- Waiting for Kijunsen Diversion...
> 01:34:26 | EUR/AUD | m1 | DownRoad(1) -- Waiting for Kijunsen Diversion...
> 01:34:26 | EUR/USD | m1 | DownRoad(1) -- Waiting for Kijunsen Diversion...
> 01:34:28 | AUD/NZD | m1 | DownRoad(1) -- Waiting for Kijunsen Diversion...
> 01:34:28 | USD/JPY | m1 | DownRoad(1) -- Waiting for Kijunsen Diversion...


|WARNING|2021-05-07 01:35:01,421|Got a insert event for closed positions: {'t': 2, 'ratePrecision': 5, 'tradeId': '242682406', 'accountName': '04290918', 'roll': 0, 'com': 0, 'open': 1.3891, 'valueDate': '', 'grossPL': -10.86, 'close': 1.39091, 'visiblePL': -18.1, 'currency': 'GBP/USD', 'isBuy': False, 'amountK': 6, 'TradeIDOrigin': None, 'OpenOrderID': '842375436', 'CloseOrderID': '842375550', 'CloseOrderReqID': 'Request-628813', 'CloseOrderRequestTXT': 'FXTC', 'CloseOrderParties': None, 'OpenOrderParties': None, 'TradeIDRemain': None, 'currencyPoint': 0.6, 'openTime': '05062021204937', 'closeTime': '05062021210501', 'action': 'I'}.
|WARNING|2021-05-07 01:35:01,426|Got a delete event for open posi: {'t': 1, 'tradeId': '242682406', 'action': 'D'}


> 01:35:20 | EUR/GBP | m1 | RoadCross(1) Found... 
> 01:35:22 | AUD/JPY | m1 | NOT MATCH | 99.38 D |  TK: 84.8560 -- KJ: 84.8585
> 01:35:22 | CAD/JPY | m1 | NOT MATCH | 96.25 D |  TK: 89.7480 -- KJ: 89.7630
> 01:35:22 | AUD/USD | m1 | NOT MATCH | 100.00 D |  TK: 0.7780 -- KJ: 0.7780
> 01:35:22 | USD/CAD | m1 | NOT MATCH | 99.91 D |  TK: 1.2146 -- KJ: 1.2149
> 01:35:22 | NZD/CAD | m1 | NOT MATCH | 99.88 D |  TK: 0.8783 -- KJ: 0.8788
> 01:35:35 | EUR/GBP | m1 | DownRoad(1) -- Waiting for Kijunsen Diversion...
> 01:36:24 | EUR/CAD | m1 | NOT MATCH | 99.91 D |  TK: 1.4651 -- KJ: 1.4655
> 01:36:24 | NZD/USD | m1 | NOT MATCH | 99.95 D |  TK: 0.7231 -- KJ: 0.7233
> 01:36:24 | CAD/CHF | m1 | NOT MATCH | 99.91 D |  TK: 0.7464 -- KJ: 0.7467
> 01:36:24 | AUD/CAD | m1 | NOT MATCH | 99.95 D |  TK: 0.9451 -- KJ: 0.9453
> 01:36:27 | AUD/JPY | m1 | NOT MATCH | 99.13 D |  TK: 84.8550 -- KJ: 84.8585
> 01:36:28 | NZD/CAD | m1 | NOT MATCH | 99.88 D |  TK: 0.8783 -- KJ: 0.8788
> 01:36:31 | AUD/JPY | m1 | N